<a href="https://colab.research.google.com/github/kachiO/TTS-examples/blob/main/Copy_of_Tacotron_DynamicConvAttn_Universal_FullBandGAN_LJSpeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mozilla TTS on CPU Real-Time Speech Synthesis 

## Tacotron2 with Dynamic Convolutional Attention
Paper: https://arxiv.org/abs/2005.11129

Trained with **LJSpeech** for **200K steps** and reduction rate of 2.

## Universal FullBand-MelGAN
Paper: https://arxiv.org/abs/2005.05106 

Trained with **LibriTTS** for **675K steps** with real spectrograms.

This model is able to synthesize speech in different languages and voices.

**Note that** TTS and vocoder models have different sampling rates. This is fixed by
interpolating TTS output before Vocoder inference.

### Download Models

In [ ]:
!gdown --id 1CFoPDQBnhfBFu2Gc0TBSJn8o-TuNKQn7 -O tts_model.pth.tar
!gdown --id 1lWSscNfKet1zZSJCNirOn7v9bigUZ8C1 -O config.json
!gdown --id 1qevpGRVHPmzfiRBNuugLMX62x1k7B5vK -O scale_stats.npy

Downloading...
From: https://drive.google.com/uc?id=1CFoPDQBnhfBFu2Gc0TBSJn8o-TuNKQn7
To: /content/tts_model.pth.tar
339MB [00:02, 149MB/s]
Downloading...
From: https://drive.google.com/uc?id=1lWSscNfKet1zZSJCNirOn7v9bigUZ8C1
To: /content/config.json
100% 11.7k/11.7k [00:00<00:00, 23.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qevpGRVHPmzfiRBNuugLMX62x1k7B5vK
To: /content/scale_stats.npy
100% 10.5k/10.5k [00:00<00:00, 14.4MB/s]


In [ ]:
!gdown --id 1i16_9_qLgtjdT9W5hfB74BlLHihReK87 -O vocoder_model.pth.tar
!gdown --id 1uBRVNxsoCYJxNCqPoQedASm6EtSW3w04 -O config_vocoder.json
!gdown --id 1JZugLTx8li1EMgptiXwh4AsOFYrJjobi -O scale_stats_vocoder.npy

Downloading...
From: https://drive.google.com/uc?id=1i16_9_qLgtjdT9W5hfB74BlLHihReK87
To: /content/vocoder_model.pth.tar
109MB [00:01, 92.7MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1uBRVNxsoCYJxNCqPoQedASm6EtSW3w04
To: /content/config_vocoder.json
100% 7.51k/7.51k [00:00<00:00, 11.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1JZugLTx8li1EMgptiXwh4AsOFYrJjobi
To: /content/scale_stats_vocoder.npy
100% 10.5k/10.5k [00:00<00:00, 15.4MB/s]


### Setup Libraries

In [ ]:
! sudo apt-get install espeak

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 16 not upgraded.
Need to get 1,219 kB of archives.
After this operation, 3,031 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsonic0 amd64 0.2.0-6 [13.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak-data amd64 1.48.04+dfsg-5 [934 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libespeak1 amd64 1.48.04+dfsg-5 [145 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 espeak amd64 1.48.04+dfsg-5 [61.6 kB]
Fetched 1,219 kB in 0s (10.5 MB/s)
de

In [ ]:
!git clone https://github.com/mozilla/TTS TTS_repo

Cloning into 'TTS_repo'...
remote: Enumerating objects: 568, done.
remote: Counting objects: 100% (568/568), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 10899 (delta 383), reused 491 (delta 341), pack-reused 10331
Receiving objects: 100% (10899/10899), 122.48 MiB | 21.19 MiB/s, done.
Resolving deltas: 100% (7587/7587), done.


In [ ]:
%cd TTS_repo
!git checkout 4132240
!pip install -r requirements.txt
!python setup.py develop
%cd ..

/content/TTS_repo
Note: checking out '4132240'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 4132240 Merge branch 'dev' of https://github.com/mozilla/TTS into dev
     |████████████████████████████████| 320.4MB 56kB/s 
     |████████████████████████████████| 1.6MB 34.6MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 235kB 42.0MB/s 
     |████████████████████████████████| 317kB 43.8MB/s 
     |████████████████████████████████| 32.4MB 111kB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
     |████████████████████████████████| 225kB 19.1MB/s 
     |█████████████

### Define TTS function

In [ ]:
def interpolate_vocoder_input(scale_factor, spec):
    """Interpolation to tolarate the sampling rate difference
    btw tts model and vocoder"""
    print(" > before interpolation :", spec.shape)
    spec = torch.tensor(spec).unsqueeze(0).unsqueeze(0)
    spec = torch.nn.functional.interpolate(spec, scale_factor=scale_factor, mode='bilinear').squeeze(0)
    print(" > after interpolation :", spec.shape)
    return spec


def tts(model, text, CONFIG, use_cuda, ap, use_gl, figures=True):
    t_1 = time.time()
    # run tts
    target_sr = CONFIG.audio['sample_rate']
    waveform, alignment, mel_spec, mel_postnet_spec, stop_tokens, inputs =\
     synthesis(model,
               text,
               CONFIG,
               use_cuda,
               ap,
               speaker_id,
               None,
               False,
               CONFIG.enable_eos_bos_chars,
               use_gl)
    # run vocoder
    mel_postnet_spec = ap._denormalize(mel_postnet_spec.T).T
    if not use_gl:
        target_sr = VOCODER_CONFIG.audio['sample_rate']
        vocoder_input = ap_vocoder._normalize(mel_postnet_spec.T)
        if scale_factor[1] != 1:
            vocoder_input = interpolate_vocoder_input(scale_factor, vocoder_input)
        else:
            vocoder_input = torch.tensor(vocoder_input).unsqueeze(0)
        waveform = vocoder_model.inference(vocoder_input)
    # format output
    if use_cuda and not use_gl:
        waveform = waveform.cpu()
    if not use_gl:
        waveform = waveform.numpy()
    waveform = waveform.squeeze()
    # compute run-time performance
    rtf = (time.time() - t_1) / (len(waveform) / ap.sample_rate)
    tps = (time.time() - t_1) / len(waveform)
    print(waveform.shape)
    print(" > Run-time: {}".format(time.time() - t_1))
    print(" > Real-time factor: {}".format(rtf))
    print(" > Time per step: {}".format(tps))
    # display audio
    IPython.display.display(IPython.display.Audio(waveform, rate=target_sr))  
    return alignment, mel_postnet_spec, stop_tokens, waveform

### Load Models

In [ ]:
import sys
import os
import torch
import time
import IPython

# for some reason TTS installation does not work on Colab
sys.path.append('TTS_repo')

from TTS.utils.io import load_config
from TTS.utils.audio import AudioProcessor
from TTS.tts.utils.generic_utils import setup_model
from TTS.tts.utils.text.symbols import symbols, phonemes
from TTS.tts.utils.synthesis import synthesis
from TTS.tts.utils.io import load_checkpoint

In [ ]:
# runtime settings
use_cuda = True

In [ ]:
# model paths
TTS_MODEL = "tts_model.pth.tar"
TTS_CONFIG = "config.json"
VOCODER_MODEL = "vocoder_model.pth.tar"
VOCODER_CONFIG = "config_vocoder.json"

In [ ]:
# load configs
TTS_CONFIG = load_config(TTS_CONFIG)
VOCODER_CONFIG = load_config(VOCODER_CONFIG)

TTS_CONFIG.audio['stats_path'] = "./scale_stats.npy"
VOCODER_CONFIG.audio['stats_path'] = "./scale_stats_vocoder.npy"


In [ ]:
# load the audio processor
ap = AudioProcessor(**TTS_CONFIG.audio)         

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:./scale_stats.npy
 | > hop_length:256
 | > win_length:1024


In [ ]:
# LOAD TTS MODEL
# multi speaker 
speakers = []
speaker_id = None
    
if 'characters' in TTS_CONFIG.keys():
    symbols, phonemes = make_symbols(**c.characters)

# load the model
num_chars = len(phonemes) if TTS_CONFIG.use_phonemes else len(symbols)
model = setup_model(num_chars, len(speakers), TTS_CONFIG)      

# load model state
model, _ =  load_checkpoint(model, TTS_MODEL, use_cuda=use_cuda)
model.eval();

 > Using model: Tacotron2
 > Model r:  2


In [ ]:
from TTS.vocoder.utils.generic_utils import setup_generator

# LOAD VOCODER MODEL
vocoder_model = setup_generator(VOCODER_CONFIG)
vocoder_model.load_state_dict(torch.load(VOCODER_MODEL, map_location="cpu")["model"])
vocoder_model.remove_weight_norm()
vocoder_model.inference_padding = 0

# scale factor for sampling rate difference
scale_factor = [1,  VOCODER_CONFIG['audio']['sample_rate'] / ap.sample_rate]
print(f"scale_factor: {scale_factor}")

ap_vocoder = AudioProcessor(**VOCODER_CONFIG['audio'])    
if use_cuda:
    vocoder_model.cuda()
vocoder_model.eval();

 > Generator Model: fullband_melgan_generator
scale_factor: [1, 1.08843537414966]
 > Setting up Audio Processor...
 | > sample_rate:24000
 | > resample:False
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:0
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:./scale_stats_vocoder.npy
 | > hop_length:256
 | > win_length:1024


## Run Inference

In [ ]:
sentence =  "Bill got in the habit of asking himself “Is that thought true?” and if he wasn’t absolutely certain it was, he just let it go."
align, spec, stop_tokedns, wav = tts(model, sentence, TTS_CONFIG, use_cuda, ap, use_gl=False, figures=True)

 > before interpolation : (80, 664)
 > after interpolation : torch.Size([1, 80, 722])


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3103: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor changed "


(184832,)
 > Run-time: 1.6879913806915283
 > Real-time factor: 0.2013635221182516
 > Time per step: 9.132157030858492e-06
